In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.offline as pyo

In [58]:
#creating dataset
matches = pd.read_csv('matches.csv')
deliveries = pd.read_csv('deliveries.csv')
ipl=deliveries.merge(matches,left_on='match_id',right_on='id')
#ipl.head()
ipl.columns

Index(['match_id', 'inning', 'batting_team', 'bowling_team', 'over', 'ball',
       'batsman', 'non_striker', 'bowler', 'is_super_over', 'wide_runs',
       'bye_runs', 'legbye_runs', 'noball_runs', 'penalty_runs',
       'batsman_runs', 'extra_runs', 'total_runs', 'player_dismissed',
       'dismissal_kind', 'fielder', 'id', 'Season', 'city', 'date', 'team1',
       'team2', 'toss_winner', 'toss_decision', 'result', 'dl_applied',
       'winner', 'win_by_runs', 'win_by_wickets', 'player_of_match', 'venue',
       'umpire1', 'umpire2', 'umpire3'],
      dtype='object')

In [24]:
#Find first 50 run scorer
top50_batsman=ipl.groupby('batsman')['batsman_runs'].sum().sort_values(ascending=False).head(50).index.to_list()

In [78]:
top50_batman_df= ipl[ipl['batsman'].isin(top50_batsman)]
top50_batman_df_grp=top50_batman_df.groupby('batsman')['batsman_runs'].agg(['sum','count'])
top50_batman_df_grp['sr']=top50_batman_df_grp['sum']/top50_batman_df_grp['count'] *100
top50_batman_df_grp.reset_index(inplace=True)
top50_batman_df_grp.head()

,batsman,sum,count,sr
0,AB de Villiers,4428,2977,148.740343
1,AC Gilchrist,2069,1555,133.054662
2,AD Russell,1445,803,179.950187
3,AJ Finch,1744,1372,127.113703
4,AM Rahane,3850,3215,119.751166


In [94]:
top50_outs=ipl[ipl['player_dismissed'].isin(top50_batsman)]['player_dismissed'].value_counts().reset_index()
top50_outs.columns=['batsman','outs']
top50_sr_avg=top50_batman_df_grp.merge(top50_outs,on='batsman')
top50_sr_avg['avg']=top50_sr_avg['sum']/top50_sr_avg['outs']
top50_sr_avg.head()




,batsman,sum,count,sr,outs,avg
0,AB de Villiers,4428,2977,148.740343,110,40.254545
1,AC Gilchrist,2069,1555,133.054662,76,27.223684
2,AD Russell,1445,803,179.950187,42,34.404762
3,AJ Finch,1744,1372,127.113703,67,26.029851
4,AM Rahane,3850,3215,119.751166,116,33.189655


In [100]:
# in plotly to create a graph we require figure object.
# this figure object takes 2 inputs trace (which is data) and layout(used to set other properties of layout)
# trace is a list of data which needs to be plotted on the graph
#scatter plot

trace1=go.Scatter(x=top50_sr_avg['avg'],y=top50_sr_avg['sr'],mode='markers')
data=[trace1]

layout=go.Layout(title='Average vs Strike Rate of top 50 Batsman',xaxis={'title':'Average of batsman'},yaxis={'title':'Strike rate'})
fig=go.Figure(data,layout)
pyo.plot(fig)


'temp-plot.html'

In [108]:
# to show batsman name on hover use text parameter, makrker parameter is used to change size & color of marker
# also we can specify the file name in plot function else it will create all file with temp-plot

trace1=go.Scatter(x=top50_sr_avg['avg'],y=top50_sr_avg['sr'],mode='markers',text=top50_sr_avg['batsman'],marker={'color':'#00a65a','size':16})
data=[trace1]

layout=go.Layout(title='Average vs Strike Rate of top 50 Batsman',xaxis={'title':'Average of batsman'},yaxis={'title':'Strike rate'})
fig=go.Figure(data,layout)
pyo.plot(fig,filename='scatterplot.html')


'scatterplot.html'